This notebook is designed for pre-iptimization of generated structures

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from pymatgen.core.structure import Structure
from pymatgen.analysis.structure_matcher import StructureMatcher
from IPython.display import clear_output
from tqdm.notebook import tqdm

In [ ]:
name = "<name of .csv file with generated structured. The file is obtained from one of *_inference.ipynb notebooks>"
dataframe = pd.read_csv(name)

Creation of array with structures

In [ ]:
structures_array = []
structures_indexes = []
num_none = 0
for i in range(dataframe.shape[0]):
    try:
        structures_array.append(Structure.from_str(dataframe.iloc[i]['cif'], fmt='cif'))
        structures_indexes.append(i)
    except:
        # structures_array.append(None)
        num_none += 1
clear_output()
num_none

Structures optimization with Pymatgen

In [ ]:
cif_array = []
mask = []

for structure in tqdm(structures_array):
    if structure.volume < 1:
        mask.append(False)
        cif_array.append(None)
        continue
    cif_array.append(
        structure.relax(
            calculator = 'm3gnet', 
            relax_cell = False, 
            stress_weight = 0.005, 
            steps = 500, 
            fmax = 0.05, 
            verbose = False
        ).to(fmt='cif')
    )
    mask.append(True)
mask = np.array(mask)

In [ ]:
dataframe["relaxed_cif"] = cif_array
print(dataframe[mask].shape)
dataframe[mask].to_csv(name, index=False)